In [210]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [211]:
# Importa la biblioteca PyTorch, que se utiliza para construir y entrenar redes neuronales.
import torch
#Importa la biblioteca NumPy, que se utiliza para operaciones numéricas eficientes en Python.
import numpy as np
#Importa la biblioteca Pandas, que se utiliza para trabajar con estructuras de datos tabulares, como DataFrames.
import pandas as pd

In [212]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/machine learning/Datasets/train data credit card.csv')
num_filas, num_columnas = data.shape

keys = data.keys()
print(keys)
print(f'Número de filas: {num_filas}\n')
print(f'Número de columnas: {num_columnas}\n')
#num_labels = data['posttest'].nunique()
#print("Número de etiquetas únicas para y:", num_labels)

Index(['ID', 'Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code',
       'Vintage', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active',
       'Is_Lead'],
      dtype='object')
Número de filas: 245725

Número de columnas: 11



In [213]:
# Eliminar las columnas innecesarias del DataFrame
#columns_to_drop = ['ID', 'CustomerId', 'Surname']
columns_to_drop = ['ID','Region_Code']
data = data.drop(columns=columns_to_drop)
# Convertir columnas categóricas en one-hot encoding
#data = pd.get_dummies(data, columns=['Geography'], prefix=['Country'])
data = pd.get_dummies(data,  columns=['Gender', 'Is_Active','Occupation','Channel_Code'])
data = pd.get_dummies(data,  columns=[ 'Credit_Product'], dummy_na=True)
# Convertir la columna "Gender" en variables one-hot
columns_to_drop2 = ['Gender_Female','Is_Active_No','Credit_Product_No']
data = data.drop(columns=columns_to_drop2)
Is_Lead   = data['Is_Lead']
# Eliminar la columna "Is_Lead" del DataFrame original
data = data.drop(columns=['Is_Lead'])

# Agregar la columna "Is_Lead" al final del DataFrame
data['Is_Lead'] = Is_Lead
keys2 = data.keys()
print(keys2)

Index(['Age', 'Vintage', 'Avg_Account_Balance', 'Gender_Male', 'Is_Active_Yes',
       'Occupation_Entrepreneur', 'Occupation_Other', 'Occupation_Salaried',
       'Occupation_Self_Employed', 'Channel_Code_X1', 'Channel_Code_X2',
       'Channel_Code_X3', 'Channel_Code_X4', 'Credit_Product_Yes',
       'Credit_Product_nan', 'Is_Lead'],
      dtype='object')


In [214]:
X, y = data.iloc[:, :15], data.iloc[:, 15]
print(X.shape)
print(y.shape)
print(X,y)

(245725, 15)
(245725,)
        Age  Vintage  Avg_Account_Balance  Gender_Male  Is_Active_Yes  \
0        73       43              1045696            0              0   
1        30       32               581988            0              0   
2        56       26              1484315            0              1   
3        34       19               470454            1              0   
4        30       33               886787            0              0   
...     ...      ...                  ...          ...            ...   
245720   51      109              1925586            1              0   
245721   27       15               862952            1              1   
245722   26       13               670659            0              0   
245723   28       31               407504            0              0   
245724   29       21              1129276            1              0   

        Occupation_Entrepreneur  Occupation_Other  Occupation_Salaried  \
0                         

In [215]:
from sklearn.model_selection import train_test_split
# Dividir los datos en entrenamiento y prueba (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Asegurarse de que las etiquetas sean de tipo entero
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
y_train = y_train.astype(np.float)
y_test = y_test.astype(np.float)

#print(X_test.info())
#print(X_train.info())

<ipython-input-215-90b5b4839da0>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train = X_train.astype(np.float)
<ipython-input-215-90b5b4839da0>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test = X_test.astype(np.float)
<ipython-input-215-90b5b4839da0>:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `fl

In [216]:
"""print("X_train:\n",X_train)
print("y_train:\n",y_train)
print("X_test:\n",X_test)
print("y_test:\n",y_test)"""

'print("X_train:\n",X_train)\nprint("y_train:\n",y_train)\nprint("X_test:\n",X_test)\nprint("y_test:\n",y_test)'

###Modelo

In [217]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Convertir los DataFrames a tensores de PyTorch
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Definir un modelo de regresión logística en PyTorch
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

# Crear una instancia del modelo
input_size = X_train.shape[1]
model = LogisticRegressionModel(input_size)

# Definir la función de pérdida y el optimizador
criterion = nn.BCELoss()
#criterion = nn.MSELoss()
#criterion = nn.NLLLoss()
#criterion = nn.KLDivLoss()


optimizer = optim.SGD(model.parameters(), lr=0.000001)

# Entrenar el modelo
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass y optimización
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluar el modelo en datos de prueba
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_binary = (y_pred >= 0.5).float()  # Convertir probabilidades a etiquetas binarias (0 o 1)
    accuracy = (y_pred_binary == y_test).float().mean()
    print(f'Accuracy en los datos de prueba: {accuracy.item():.4f}')



Epoch [10/100], Loss: 23.7374
Epoch [20/100], Loss: 23.7374
Epoch [30/100], Loss: 23.7374
Epoch [40/100], Loss: 23.7374
Epoch [50/100], Loss: 23.7374
Epoch [60/100], Loss: 23.7374
Epoch [70/100], Loss: 23.7374
Epoch [80/100], Loss: 23.7374
Epoch [90/100], Loss: 23.7374
Epoch [100/100], Loss: 23.7374
Accuracy en los datos de prueba: 0.7635


In [218]:
#Predicciones en los datos de entrenamiento:
with torch.no_grad():
    y_pred_train = model(X_train)
    test_loss = criterion(y_pred_train, y_train.view(-1, 1))
print(f'Loss en los datos de prueba: {test_loss.item():.4f}')
#Predicciones en los datos de prueba:
with torch.no_grad():
    y_pred_test = model(X_test)
    test_loss = criterion(y_pred_test, y_test.view(-1, 1))
print(f'Loss en los datos de prueba: {test_loss.item():.4f}')


Loss en los datos de prueba: 23.7374
Loss en los datos de prueba: 23.6545
